In [1]:
import art
from art.utils import load_mnist

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Flatten, InputLayer, Reshape

from art.estimators.classification import KerasClassifier

if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()
    #tf.python.framework.ops.disable_eager_execution()

c:\users\camila\appdata\local\programs\python\python38\lib\site-packages\art\estimators\certification\__init__.py:12: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [3]:
# Recrea exactamente el mismo modelo solo desde el archivo
malware_model = keras.models.load_model('malware-listo-model.h5')

In [4]:
malware_model

In [5]:
classifier_original = KerasClassifier(malware_model, clip_values=(0, 1), use_logits=False)

In [6]:
# Se cargan los datos
X_train = pd.read_csv('x_train.csv')
X_test = pd.read_csv('x_test.csv')

y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('y_test.csv')

In [7]:
# Stealing from the unprotected classifier.
from art.attacks import ExtractionAttack
from art.attacks.extraction import CopycatCNN, KnockoffNets

len_steal = 5000
num_epochs = 10
attack_catalogue = {"Probabilistic CopycatCNN": CopycatCNN(classifier=classifier_original,
                                              batch_size_fit=64,
                                              batch_size_query=64,
                                              nb_epochs=num_epochs,
                                              nb_stolen=len_steal,
                                              use_probability=True),
                    "Argmax CopycatCNN": CopycatCNN(classifier=classifier_original,
                                              batch_size_fit=64,
                                              batch_size_query=64,
                                              nb_epochs=num_epochs,
                                              nb_stolen=len_steal,
                                              use_probability=False),
                    "Probabilistic KnockoffNets": KnockoffNets(classifier=classifier_original,
                                              batch_size_fit=64,
                                              batch_size_query=64,
                                              nb_epochs=num_epochs,
                                              nb_stolen=len_steal,
                                              use_probability=True),
                    "Argmax KnockoffNets": KnockoffNets(classifier=classifier_original,
                                              batch_size_fit=64,
                                              batch_size_query=64,
                                              nb_epochs=num_epochs,
                                              nb_stolen=len_steal,
                                              use_probability=False),
                   }

In [11]:
im_shape = X_train.shape
def get_model(num_classes=10, c1=32, c2=64, d1=128):
    model = Sequential()
    model.add(Conv2D(c1, kernel_size=(3, 3), activation='relu', input_shape=im_shape))
    model.add(Conv2D(c2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(d1, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer="sgd",
                          metrics=['accuracy'])
    return model

In [12]:
results = []
for len_steal in [250, 500, 1000, 2000, 4000, 5000]:
    indices = np.random.permutation(len(X_train))
    x_steal = X_train[indices[:len_steal]]
    y_steal = y_train[indices[:len_steal]]
    x_test = X_train[indices[len_steal:]]
    y_test = y_train[indices[len_steal:]]

    for name, attack in attack_catalogue.items():
        model_stolen = get_model(num_classes=10, c1=32, c2=64, d1=128)
        classifier_stolen = KerasClassifier(model_stolen, clip_values=(0, 1), use_logits=False)
        classifier_stolen = attack.extract(x_steal, y_steal, thieved_classifier=classifier_stolen)
        acc = classifier_stolen._model.evaluate(x_test, y_test)[1]
        print(name, ":", acc)
        results.append((name, len_steal, acc))

KeyError: "None of [Int64Index([20046600, 65849522,  9212231, 65714191, 23248589, 77292008,\n            25768162, 50392713, 55250820, 62581377,\n            ...\n            74099713, 43509055, 60505955,  1122614, 45600911, 56512513,\n            58013431, 30797567, 62901279, 28494658],\n           dtype='int64', length=250)] are in the [columns]"